In [195]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import simplefix
from simplefix import FixMessage, FixParser
import socket
import ssl

In [196]:
# Setup credentials for connection to cTrader
server_url = "h35.p.ctrader.com"
server_port = 5211

sender = "demo.ctrader.3988979"
target = "CSERVER"
username = "3988979"
password = "SamaTrades123!"

In [216]:
# Create the FIX message parser
parser = simplefix.FixParser()

# Create the FIX message
message = simplefix.FixMessage()
message.append_pair(8, "FIX.4.4")
message.append_pair(9, 126)
message.append_pair(35, "A")
message.append_pair(49, sender)
message.append_pair(56, target)
message.append_pair(34, 1, header=True)
message.append_utc_timestamp(52)
message.append_pair(57, "QUOTE")
message.append_pair(98, 0)
message.append_pair(108,30)
message.append_pair(141, "Y")
message.append_pair(553,username)
message.append_pair(554,password)
message.append_pair(10, 131)

# Convert the FIX message to bytes
message_bytes = message.encode()

In [219]:
context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    with context.wrap_socket(sock, server_hostname=server_url) as ssock:
        # Create a socket connection and send the logon FIX message
        try:
            # Connect to the backend
            ssock.connect((server_url, server_port))
            print(f"Connected to {server_url}:{server_port}")
            # Send the message bytes to the backend
            ssock.send(message_bytes)
            # Fetch the API response
            response = ssock.recv(1024)
            # Use the parser to decode the API response
            parser.append_buffer(response)
            message = parser.get_message()
            if message:
                print(f"Message received: {message}")
            else:
                print("Failed to parse the response as a valid FIX message.")

        except ssock.error as e:
            print(f"Socket error: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

Connected to h35.p.ctrader.com:5211
Message received: 8=FIX.4.4|9=97|35=A|34=1|49=CSERVER|50=QUOTE|52=20231207-11:29:18.917|56=demo.ctrader.3988979|98=0|108=30|141=Y|10=241
